In [ ]:
%matplotlib inline

In [ ]:
!pip install pandas
import pandas as pd

!pip install matplotlib
import matplotlib.pyplot as plt

!pip install numpy
import numpy as np

plt.style.use('seaborn-talk')
plt.rcParams['figure.figsize'] = (15,10)
plt.style.use('ggplot')

# Задача про датчики на руках

В этом задании мы применим метод главных компонент на многомерных данных и постараемся найти оптимальную размерность признаков для решения задачи классификации

Исходными данными являются показания различных сенсоров, установленных на руках человека, который умеет общаться на языке жестов.

В данном случае задача ставится следующим образом: по показаниям датчиков (по 11 сенсоров на каждую руку) определить слово, которое было показано человеком.

Как можно решать такую задачу?

Показания датчиков представляются в виде временных рядов. Посмотрим на показания для одного из "слов"

In [ ]:
# Загружаем данные сенсоров
df_database = pd.read_csv('https://raw.githubusercontent.com/vadim0912/MLIntro2021/main/lecture06/data/sign_database.csv')

# Загружаем метки классов
sign_classes = pd.read_csv('https://raw.githubusercontent.com/vadim0912/MLIntro2021/main/lecture06/data/sign_classes.csv', index_col=0, header=0, names=['id', 'class'])

In [ ]:
# Столбец id - идентификаторы "слов"
# Столбец time - метка времени
# Остальные столбцы - показания серсоров для слова id в момент времени time

df_database.head()

In [ ]:
# Выберем одно из слов с идентификатором = 0
sign0 = df_database.query('id == 0')\
                   .drop(['id'], axis=1)\
                   .set_index('time')

In [ ]:
sign0.plot()

Для каждого из "слов" у нас есть набор показаний сенсоров с разных частей руки в каждый момент времени.

Идея нашего подхода будет заключаться в следующем – давайте для каждого сенсора составим набор характеристик (например, разброс значений, максимальное, минимальное, среднее значение, количество "пиков", и т.п.) и будем использовать эти новые "признаки" для решения задачи классификации.

In [ ]:
!pip install scikit-learn
from sklearn.preprocessing import LabelEncoder

In [ ]:
!wget https://www.dropbox.com/s/x6b9mqxlw5ijcuf/tsfresh_features_filt.csv.gz?dl=0 -O tsfresh_features_filt.csv.gz
filepath = 'tsfresh_features_filt.csv.gz'
sign_features_filtered = pd.read_csv(filepath)

In [ ]:
sign_features_filtered.shape

In [ ]:
sign_features_filtered.head()

# Базовая модель

В результате у нас получилось очень много признаков (11516 или больше), давайте применим метод главных компонент, чтобы получить сжатое признаковое представление, сохранив при этом предиктивную силу в модели.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Создадим бейзлайн без уменьшения размерности. Гиперпараметры модели подбирались произвольно

In [ ]:
# Подготовим данные на вход в модель
X = sign_features_filtered.values
y = sign_classes.values

In [ ]:
# Будем делать кросс-валидацию на 5 фолдов
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

base_model = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', KNeighborsClassifier(n_neighbors=9))
])

base_cv_scores = cross_val_score(base_model, X, y, cv=cv, scoring='accuracy')

In [ ]:
base_cv_scores.mean()

### Пайплайн с PCA

In [ ]:
scaler = StandardScaler()
Z = scaler.fit_transform(X)

In [ ]:
pca = PCA(svd_solver='randomized', random_state=123)
Z = pca.fit_transform(Z)

In [ ]:
ob_dis = pca.explained_variance_ratio_

Рассматриваем только те дисперсии, которые превышают значение 1e-3, потому что дальше, чтобы увеличить кол-во информации на условную единицу надо прибавлять больше компонент

In [ ]:
upper1e3 = np.cumsum(ob_dis[np.where(ob_dis > 1e-3)])

print(f'Количество информации в первых {len(upper1e3)} компонентах: {upper1e3[-1]}')

In [ ]:
# Будем делать кросс-валидацию на 5 фолдов
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

model_with_pca = Pipeline([
    ('scaler', StandardScaler()),
    ('PCA', PCA(svd_solver='randomized', random_state=123, n_components=len(upper1e3))),
    ('clf', KNeighborsClassifier(n_neighbors=9))
])

pca_cv_scores = cross_val_score(model_with_pca, X, y, cv=cv, scoring='accuracy')

In [ ]:
pca_cv_scores.mean()

0.9115009746588694

In [ ]:
using_dis = ob_dis[np.where(ob_dis > 1e-3)]

fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.scatter(range(ob_dis.shape[0]), ob_dis)
ax1.scatter(range(using_dis.shape[0]), using_dis)

ax2.scatter(range(using_dis.shape[0]), using_dis, c='g')

ax1.legend( ['Не использованная объяснённая дисперсия','Доля объяснённой дисперсии при найденной настройке PCA'])

ax2.legend(['Доля объяснённой дисперсии при найденной настройке PCA'], loc=10)

# Задача про кластеризацию текстов

Рассмотрим коллекцию новостных сообщений за первую половину 2017 года. Про каждое новостное сообщение известны:

* его заголовок и текст
* дата его публикации
* событие, о котором это новостное сообщение написано
* его рубрика


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/vadim0912/MLIntro2021/main/lecture06/data/news.csv', encoding='utf8')
df.head()

In [ ]:
df.loc[:, 'class'].value_counts()

Попробуем кластеризовать документы (каким-либо методом) и сравним полученное разбиение с данными рубликами с помощью ARI

### Стандартная предобработка текстов

Ниже выполняется набор операций по предобработке текстов.

In [ ]:
import re

# Оставляем только кириллические символы
regex = re.compile(u"[А-Яа-я]+")

def words_only(text, regex=regex):
    return " ".join(regex.findall(text))


df.text = df.text.str.lower()
df.loc[:, 'text'] = df.text.apply(words_only)

In [ ]:
!pip install nltk
import nltk

In [ ]:
from nltk.corpus import stopwords

nltk.download('stopwords')

# Удаляем стоп-слова
mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', u'также',  'т', 'д', '-', '-']

def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return u" ".join([token for token in text.split() if not token in mystopwords])
    except:
        return u""
    
df.text = df.text.apply(remove_stopwords)   

In [ ]:
!pip install pymystem3
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz

In [ ]:
%%time 
from pymystem3 import Mystem


# нормализуем текст
bin_path = "./mystem"
m = Mystem(bin_path)
def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

df.text = df.text.apply(lemmatize)

In [ ]:
mystoplemmas = [u'который', u'прошлый', u'сей', u'свой', u'наш', u'мочь']

# Еще кое-что удаляем
def  remove_stoplemmas(text, mystoplemmas = mystoplemmas):
    try:
        return " ".join([token for token in text.split() if not token in mystoplemmas])
    except:
        return ""

df.text = df.text.apply(remove_stoplemmas)  

### Вычисление сходства (1 балл)

In [ ]:
!pip install seaborn
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances

In [ ]:
vect = TfidfVectorizer()
texts = df.text.values

In [ ]:
tfidf = vect.fit_transform(texts)

In [ ]:
S = pairwise_distances(tfidf, metric='cosine')

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(data=S, cmap = 'Spectral').set(xticklabels=[],yticklabels=[])

### DBSCAN (4 балла)

* (2) Воспользуйтесь методикой оценки параметров для алгоритма DBSCAN. Не копипастите min_pts = 2 из семинара! Используйте косинусную меру близости.
* (1) Выделите кластеры. Для каждого кластера (кроме -1, если он будет) выведите несколько текстов и умозрительно определите его тематику. Можете подсмотреть в исходные тематики корпуса
* (1) Оцените сходство с изначальными рубриками визуально (с помощью матрицы перемешивания) и с помощью Adjusted Rand Index

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

In [ ]:
min_samples = 5
nn = NearestNeighbors(n_neighbors=min_samples)
nn.fit(tfidf)

In [ ]:
dist , _ = nn.kneighbors(tfidf)
dist_last = dist[:, -1]
dist_last = np.sort(dist_last)

In [ ]:
plt.plot(dist_last);

In [ ]:
db_scan = DBSCAN(eps=0.66, min_samples=6, metric='cosine')
db_scan.fit(tfidf)

In [ ]:
df_ = pd.read_csv('https://raw.githubusercontent.com/vadim0912/MLIntro2021/main/lecture06/data/news.csv', encoding='utf8')
df_.loc[:, 'label'] =  db_scan.labels_

In [ ]:
df_.label.value_counts()

In [ ]:
for cluster, df_cluster in df_.groupby('label'):
    print(f'== Cluster {cluster} ==')
    print(f'Cluster size: {df_cluster.shape[0]}')
    df_cluster.drop(columns=["label"]).mean()
    print(df_cluster.text[:5])
    print()

После кластеризации явно выделяются следующие кластеры:
* Туризм
* Спорт
* Технологии
* Катастрофы (теракты)

Новости связанные с политикой перемешиваются, но есть некоторые кластеры:
* Культура
* Опозиция
* Выборы
* Евровидение


In [ ]:
labels = db_scan.labels_

In [ ]:
pd.crosstab(df.loc[:, 'class'], labels)

In [ ]:
from sklearn.metrics.cluster import adjusted_rand_score

In [ ]:
le = LabelEncoder()
true_label = le.fit_transform(df['class'])
adjusted_rand_score(true_label, labels)